Notebook

# Climate Displacement

Disaster Types
- Year
- State
- Households Inflow (Number of Returns)
- Households Outflow (Number of Returns)
- Individuals Inflow (Number of Exemptions)
- Individuals Outflow (Number of Exemptions)
- Chemical
- Dam/Levee Break
- Drought
- Earthquake
- Fire
- Flood
- Human Cause
- Hurricane
- Ice
- Mud/Landslide
- Other
- Snow
- Storm
- Terrorism
- Tornado
- Tsunami
- Typhoon
- Volcano
- Water
- Winter

In [ ]:
"""
Folders Setup

code 
    notebook.ipynb
    data
        Disasters
            FEMA_dataset.csv
        StateMigration
            1990to1991StateMigration
                 1990to1991StateMigrationInflow
                     Alabama91in.xls
                     Alaska91in.xls
                     .
                     .
                     .
                     Wisconsin91in.xls
                     Wyoming91in.xls 
                 1990to1991StateMigrationOutflow
                     Alabama91Out.xls
                     Alaska91Out.xls 
                     .
                     .
                     .
                     Wisconsin91Out.xls
                     Wyoming91Out.xls 
            .
            .
            .
            2008to2009StateMigration
            2009to2010StateMigration
            2010to2011StateMigration 
"""

## Pseudocode:

### FEMA Dataset Pre-processing
1. Create new FEMA_dataset with columns 
    - contains Year, State, Disaster Type
2. Name file "State_Disasters_by_Year"

### StateMigration Data Pre-Processing
1. Convert all datasets in StateMigration from .xls into .csv files
2. Extract "Total Flow" row with "Number of Returns" and "Number of Exemptions" - assign I if from inflow and O if from outflow - from every state file.
3. Extract "State" and "Year" from every file
4. Create file with "State" (from file name), "Year" (from file name), "Number_of_Returns_I", "Number_of_Exemptions_I", "Number_of_Returns_I" and "Number_of_Exemptions_O"
5. Name file "State Migration by Year"

### Merge Datasets
1. Merge datasets on common attributes "Year" and "State"
2. Name dataset "State_Migration_and_Disasters_by_Year"


```
read_file = pd.read_excel ("Test.xlsx")
 
# Write the dataframe object
# into csv file
read_file.to_csv ("Test.csv",
                  index = None,
                  header=True)
```

In [43]:
# adding imports
import shutil
import pandas as pd
import os
import glob
import io
import xlrd
import csv

FEMA Dataset Pre-processing

Creates State_Disasters_by_Year.csv with:
- State
- Disaster Type
- Start Year
- End Year

In [ ]:
# FEMA Dataset Preprocessing

# copy original FEMA dataset to new file
original = r'../code/data/Disasters/FEMA_dataset.csv'
new = r'../code/data/Disasters/State_Disasters_by_Year.csv'
shutil.copyfile(original, new)

# read csv file
data = pd.read_csv('~/code/data/Disasters/State_Disasters_by_Year.csv')

# delete irrelevant rows
data.pop('Declaration Number')
data.pop('Declaration Type')
data.pop('Declaration Date')
data.pop('County')
data.pop('Disaster Title')
data.pop('Close Date')
data.pop('Individual Assistance Program')
data.pop('Public Assistance Program')
data.pop('Hazard Mitigation Program')
data.pop('Individuals & Households Program')

# extract years
data['Start Year'] = pd.DatetimeIndex(data['Start Date']).year
data['End Year'] = pd.DatetimeIndex(data['End Date']).year

# delete start and end dates
data.pop('Start Date')
data.pop('End Date')

print(data)

# save changes csv
data.to_csv('../code/data/Disasters/State_Disasters_by_Year.csv')

In [58]:
# Convert all files in StateMigration folder from .xls to .csv

# create list of xls files
xls_list = glob.glob("/Users/ben/Desktop/climate-displacement/code/data/StateMigration/*/*/*.xls")

# replace xls 
for xls_file in xls_list:
    print(xls_file[-20:-3])
    wb = xlrd.open_workbook(xls_file)
    sh = wb.sheet_by_index(0)
    csv_file = open(xls_file[0:-3]+'csv', "w")
    wr = csv.writer(csv_file, quoting=csv.QUOTE_ALL)
    
    for rownum in range(sh.nrows):
        wr.writerow(sh.row_values(rownum))
        
    csv_file.close()
    #read_file = pd.read_table(xls_file)
    #read_file.to_csv(xls_file[0:-3]+'csv', index = None, header=True, sep='')
    #read_xls
    #DataFrame
    #print to_csv
    
    #os.remove(xls_file)


nInflow/SoCa04in.
nInflow/Idah04in.
nInflow/Loui04in.
nInflow/wisc04in.
nInflow/Okla04in.
nInflow/DiCo04in.
nInflow/Dela04in.
nInflow/Mich04in.
nInflow/Nebr04in.
nInflow/Conn04in.
nInflow/RhIs04in.
nInflow/Virg04in.
nInflow/Miso04in.
nInflow/Verm04in.
nInflow/Alas04in.
nInflow/Penn04in.
nInflow/SoDa04in.
nInflow/Oreg04in.
nInflow/Mont04in.
nInflow/Main04in.
nInflow/NoDa04in.
nInflow/NeHa04in.
nInflow/NeMe04in.
nInflow/Alab04in.
nInflow/Indi04in.
nInflow/Wash04in.
nInflow/Utah04in.
nInflow/Neva04in.
nInflow/Illi04in.
nInflow/Iowa04in.
nInflow/Mass04in.
nInflow/Ohio04in.
nInflow/Colo04in.
nInflow/Geor04in.
nInflow/Cali04in.
nInflow/Hawa04in.
nInflow/Tenn04in.
nInflow/Wyom04in.
nInflow/Flor04in.
nInflow/Ariz04in.
nInflow/Kent04in.
nInflow/NoCa04in.
nInflow/Arka04in.
nInflow/Kans04in.
nInflow/wevi04in.
nInflow/Texa04in.
nInflow/Minn04in.
nInflow/NeJe04in.
nInflow/Mary04in.
nInflow/NeYo04in.
nInflow/Misi04in.
Outflow/utah04ot.
Outflow/wash04ot.
Outflow/colo04ot.
Outflow/ohio04ot.
Outflow/ma

XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b'ARIZONA '

In [ ]:
# converting 2003-04 State Migration from xls to csv
xls_list = glob.glob("../code/data/StateMigration/2003to2004StateMigration/*/*")

for xls_file in xls_list:
    read_file = pd.read_table(xls_file)
    
    #read_file = pd.read_excel(xls_file)
    #read_file.to_csv(xls_file[0:-3]+'csv', encoding='utf-8')
    #open_wb = xlrd.open_workbook(xls_file)
    #c_csv = open(xls_file[0:-3]+'csv', open_wb)
    
    #write_csv = csv.writer(c_csv, quoting=csv.QUOTE_ALL)
    #c_csv.close()
    #print(xls_file) 

In [ ]:
"""
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f) and filename.endswith('.txt'):
        read_file = pd.read_excel (r'../code/data/StateMigration/1990to1991StateMigration/1990to1991StateMigrationInflow/Alabama91in.xls')
    read_file.to_csv (r'../code/data/StateMigration/1990to1991StateMigration/1990to1991StateMigrationInflow/Alabama91in.csv', index = None, header=True)


read_file = pd.read_excel (r'../code/data/StateMigration/1990to1991StateMigration/1990to1991StateMigrationInflow/Alabama91in.xls')
read_file.to_csv (r'../code/data/StateMigration/1990to1991StateMigration/1990to1991StateMigrationInflow/Alabama91in.csv', index = None, header=True)



location = "/Users/neely/Desktop/climate-displacement/code/data/StateMigration/1990to1991StateMigration/1990to1991StateMigrationInflow"
placement = "./Users/neely/Desktop/climate-displacement/code/data/StateMigration"

for file in glob.glob("*.xls"):
"""

"""
from os.path import isfile, join
onlyfiles = [f for f in listdir('../code/data/StateMigration/') if isfile(join('../code/data/StateMigration/', f))]
"""